In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
import datetime
import numpy as np
import time
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.document_transformers import LongContextReorder
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH00-RAG-Chatbot")

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434" # Ollama 서버 주소
OLLAMA_CHAT_MODEL = "ollama-ko-0710:latest" # Ollama에서 실행중인 모델 이름

In [ ]:
def get_split_docs():
    # 1. 텍스트 청크 분할 (chunk_size=1000, chunk_overlap=50)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    # 2. Text loader
    loader = TextLoader("./data/text.txt", encoding="utf-8")
    return loader.load_and_split(text_splitter)

In [ ]:
def get_retriever(split_docs): 
    model_name = "intfloat/multilingual-e5-large-instruct"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda"},  # cuda, cpu
        encode_kwargs={"normalize_embeddings": True},
    )
    db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
    )
    return retriever

In [ ]:
def get_reranker(retriever):
    reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=reranker, top_n=6)
    return ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )

In [ ]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs

In [ ]:
def get_prompt():
    template = """당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
    You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.
    모든 대답은 반드시 한국말로 대답해주세요.
    
    문서 내용:
    {context}
    질문: {question}
    답변:"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    return prompt

In [ ]:
def multi_query_retriever(retriever):
    # 다중 질의어 생성
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=retriever,
    llm=llm,
    )
    return multiquery_retriever

## ReRank+ ReOrder  추가 버전

In [ ]:
def general_llm():

    import logging
    logging.basicConfig()
    logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

    split_docs = get_split_docs()

    retriever = get_retriever(split_docs)

    multi_retriever = multi_query_retriever(retriever)

    compression_retriever = get_reranker(multi_retriever)
    
    prompt= get_prompt()
    
    
    # 7. Ollama LLM 초기화
    # llm = OllamaLLM(model=OLLAMA_CHAT_MODEL, base_url=OLLAMA_BASE_URL,temperature=1, max_tokens=1024)
    llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.7, max_tokens=1024)
    chain = (
    {
        "context": itemgetter("question")
        | compression_retriever
        | RunnableLambda(reorder_documents),  # 질문을 기반으로 문맥을 검색합니다.
        "question": itemgetter("question"),  # 질문을 추출합니다.
    }
    | prompt  # 프롬프트 템플릿에 값을 전달합니다.
    | llm # 언어 모델에 프롬프트를 전달합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
    )

    answer = chain.invoke({"question": "삼성전자 하반기 실적 전망은?"})
    return answer

In [ ]:
general_llm()

